<a href="https://colab.research.google.com/github/Eddiebee/AI-Craft/blob/main/PyTorch_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook aims to gain familiarity with the PyTorch API for common Machine Learning tasks.

### Working with Data

The PyTorch API exposes two primitives to use while working with data; `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`.

`Dataset` stores the samples and their corresponding labels, whereas `DataLoader` wraps an iterable around the `Dataset`.

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In this notebook we'll be using the FashionMNIST dataset. Every TorchVision `Dataset` includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

In [3]:
# Downloading training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),)

# Downloading test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),)

100%|██████████| 26421880/26421880 [00:00<00:00, 34655555.57it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 4571450.61it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 43299592.18it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11684132.57it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



We pass the `Dataset` as argument to the `DataLoader`. The `DataLoader` wraps an iterable over the `Dataset` which enables it to support features as:

- automatic batching
- sampling
- shuffling and
- multiprocess data loading

Defining a batch size of 64 means that each element in the dataloader will return a batch of 64 features and labels.

In [4]:
batch_size = 64

# Create dataloaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models

To create a neural network in PyTorch, we create a class that inherits from `nn.Module`. We define the layers of the network in the `__init__` function and then specify how data will pass through the network in the `forward` function. In order to accelerate operations in the neural network, we move it to the GPU or MPU if available.

In [5]:
# Get cpu, gpu, or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


In [6]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the model parameters

To train a model, we need a `loss function` and an `optimizer`.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes prediction on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model's parameters.

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure that the model is actually learning.

In [11]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

In [12]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.173894  [    0/60000]
loss: 2.159714  [ 6400/60000]
loss: 2.107666  [12800/60000]
loss: 2.128518  [19200/60000]
loss: 2.071312  [25600/60000]
loss: 2.022725  [32000/60000]
loss: 2.047586  [38400/60000]
loss: 1.969570  [44800/60000]
loss: 1.976703  [51200/60000]
loss: 1.913010  [57600/60000]
Test Error: 
 Accuracy: 57.9%, Avg loss: 1.904414 

Epoch 2
-------------------------------
loss: 1.939125  [    0/60000]
loss: 1.900994  [ 6400/60000]
loss: 1.790372  [12800/60000]
loss: 1.836468  [19200/60000]
loss: 1.720502  [25600/60000]
loss: 1.683092  [32000/60000]
loss: 1.705800  [38400/60000]
loss: 1.600963  [44800/60000]
loss: 1.626991  [51200/60000]
loss: 1.524796  [57600/60000]
Test Error: 
 Accuracy: 61.2%, Avg loss: 1.535881 

Epoch 3
-------------------------------
loss: 1.604806  [    0/60000]
loss: 1.559010  [ 6400/60000]
loss: 1.412096  [12800/60000]
loss: 1.490649  [19200/60000]
loss: 1.366406  [25600/60000]
loss: 1.366222  [32000/600

Let's experiment to see how more epoch influences the model's accuracy.

In [13]:
epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 1.035970  [    0/60000]
loss: 1.042771  [ 6400/60000]
loss: 0.852580  [12800/60000]
loss: 1.022548  [19200/60000]
loss: 0.899437  [25600/60000]
loss: 0.926473  [32000/60000]
loss: 0.979976  [38400/60000]
loss: 0.915111  [44800/60000]
loss: 0.954139  [51200/60000]
loss: 0.895326  [57600/60000]
Test Error: 
 Accuracy: 66.8%, Avg loss: 0.904563 

Epoch 2
-------------------------------
loss: 0.947776  [    0/60000]
loss: 0.973231  [ 6400/60000]
loss: 0.770880  [12800/60000]
loss: 0.957445  [19200/60000]
loss: 0.840746  [25600/60000]
loss: 0.860068  [32000/60000]
loss: 0.924568  [38400/60000]
loss: 0.865762  [44800/60000]
loss: 0.898522  [51200/60000]
loss: 0.848508  [57600/60000]
Test Error: 
 Accuracy: 68.0%, Avg loss: 0.853835 

Epoch 3
-------------------------------
loss: 0.881508  [    0/60000]
loss: 0.923009  [ 6400/60000]
loss: 0.711443  [12800/60000]
loss: 0.910252  [19200/60000]
loss: 0.799952  [25600/60000]
loss: 0.811469  [32000/600

### Saving Models

A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [14]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading Models

The process for loading a model involves re-creating the model structure and loading the state dictionary into it.

In [15]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

The model can now be used to make predictions.

In [17]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [18]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
